In [66]:

## based on 
# https://danijar.com/building-variational-auto-encoders-in-tensorflow/
# https://towardsdatascience.com/teaching-a-variational-autoencoder-vae-to-draw-mnist-characters-978675c95776
## Imports

import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn.model_selection
import os, glob, shutil, math, time
# reset tf graph  
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)
#sess = tf.Session()
session = tf.InteractiveSession()
## data pipeline from R-prepared CSV

df = pd.read_csv("split_x.csv",sep=",")
data = df.values
odf = pd.read_csv("split_y.csv",sep=",")
outcomes = odf.values


## seed 
np.random.seed(134)

m = np.shape(data)[0]
d = np.shape(data)[1]
p = 1

## Functions
def data_rescale(scaled_dat,train_mean,train_var):
    d = np.shape(train_mean)[0]
    print('unnormalizing with number of dimensions = ' +str(d))
    dat = (np.multiply(scaled_dat.T,np.sqrt(train_var),)+train_mean).T
    return(dat)
def data_normalize(data,train_mean,train_var):
    d = np.shape(train_mean)[0]
    print('normalizing with number of dimensions = ' +str(d))
    scaled_dat = np.divide((data.T - train_mean),np.sqrt(train_var),).T
    return(scaled_dat)
    
## data generation
# get random partition
msk = np.random.rand(np.shape(data)[0]) < 0.01
print('msk[8] = ' + str(msk[8]))

# test and train split
train_data_x = data[msk]
test_data_x = data[~msk]
train_data_y = outcomes[msk]
test_data_y = outcomes[~msk]



# non-dimenisionalization
train_var_x = np.var(train_data_x,0).reshape(d,1)
train_mean_x = np.mean(train_data_x,0).reshape(d,1)
scaled_train_data_x = data_normalize(train_data_x,train_mean_x,train_var_x)
scaled_test_data_x = data_normalize(test_data_x,train_mean_x,train_var_x)

train_var_y = np.var(train_data_y,0).reshape(p,1)
train_mean_y = np.mean(train_data_y,0).reshape(p,1)
scaled_train_data_y = data_normalize(train_data_y,train_mean_y,train_var_y)
scaled_test_data_y = data_normalize(test_data_y,train_mean_y,train_var_y)

scaled_train_data_x = np.array([[1,2,3,1],[2,3,4,1],[6,1,2,1]])
scaled_test_data_x = np.array([[9,7,8,2],[8,2,2,1]])
d = 3
n_input = d # data input (img shape: 28*28)
n =  np.shape(scaled_train_data_x)[0]
ntest =  np.shape(scaled_test_data_x)[0]
## tf variable placeholders for feeding
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32,[None,n_input],name="X")
    Y = tf.placeholder(tf.float32,[None,n_input],name="Y")
    global_step = tf.Variable(0, trainable=False)
    keep_prob = tf.placeholder(dtype=tf.float32, shape=(), name='keep_prob')

print(scaled_train_data_x.shape)

msk[8] = False
normalizing with number of dimensions = 155
normalizing with number of dimensions = 155
normalizing with number of dimensions = 1
normalizing with number of dimensions = 1
(3, 4)


In [67]:
rA = tf.reshape(tf.reduce_sum(tf.square(scaled_train_data_x), 1),[-1,1])

# Initialize variables
init = tf.global_variables_initializer()

#sess.run(init)

#print(sess.run(X, feed_dict={X: scaled_train_data_x}))
#vv = sess.run(rA)
#vv

In [78]:
def myPairwiseDist(X,Y):
    tile1 = tf.tile(tf.expand_dims(X, 0), [Y.shape[0], 1, 1])
    tile2 = tf.tile(tf.expand_dims(Y, 1), [1,X.shape[0] , 1])
    pairwiseEuclideanDistance = tf.reduce_sum(tf.square(tf.subtract(tile1, tile2)), 2)
    return pairwiseEuclideanDistance
def myRBFKernel(gamma,X,Y):
    K = tf.exp(tf.multiply(-1*gamma, myPairwiseDist(X,Y)))
    return K

In [91]:
gamma = tf.Variable(0.5, tf.float32)
tf.initialize_all_variables().run()
gamma.eval()

0.5

In [92]:
#x_ = tf.expand_dims(scaled_train_data_x, 2) #shape 8 1 155
#xy = tf.expand_dims(scaled_test_data_x, 2) # shaoe 1439 1 155

pairwise_euclidean_distance = myPairwiseDist(scaled_train_data_x,scaled_train_data_x)
gamma = tf.Variable(0.5)
K  = myRBFKernel(gamma,scaled_train_data_x,scaled_train_data_x)
print(pairwise_euclidean_distance)
print(pairwise_euclidean_distance.eval())
print(K)
print(K.eval())

TypeError: Input 'y' of 'Mul' Op has type int32 that does not match type float32 of argument 'x'.

In [ ]:
y_ = K.expand_dims(E, 2)
  # shape=(8, 4, 4, 30, 30)

distances = tf.norm(s, axis=[-2, -1])  # shape=(8, 4, 4)
P = K.sum(distances, axis=[-2, -1])  # shape=(8,)

In [16]:
np.dot(scaled_train_data_x[0],scaled_train_data_x[0])

495.74389668522923

In [20]:
tf.add(
    tf.subtract(dist,
                tf.multiply(2., 
                            tf.matmul(x_data, tf.transpose(x_data))))
    , tf.transpose(dist))

<tf.Tensor 'Reshape:0' shape=(1151, 1) dtype=float64>

In [ ]:

rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))